Week3-Pratical
Group members:

**Yaohui WANG / Mauricia Fleurance EVINA / Hanrui ZHANG**
This is our complementory work respect to the deadline extension. In this work we just try to redo the work to create a sample of user_profile.csv cause we failed in our last submission. 
We will also try to use this sample file to to the recommender system which we just used ratings to finish the question previously.     

So **<font color = red>this file is just an additional work</font>**  and the precious question should be find in our submission **<font color = blue>before 22th Jun</font>**



In [2]:
import pandas as pd
import os
import numpy as np
#!pip install tqdm

In [3]:
#Load "genome-tags" CSV file
file_path1 = '/Users/mac/Desktop/MCGILL/RS/ml-25m/'
#file_path1 = 'C:\\Users\\hzhang245\\Desktop\\ml-25m\\'
rating_df = pd.read_csv(file_path1+'ratings.csv')

### Here I just loaded the item_profiles.csv wich already created previously.

In [4]:
item_profiles_df = pd.read_csv(file_path1+'item_profiles.csv',index_col='movieId')

In [5]:
item_profiles_df

,Thriller,Film-Noir,Fantasy,Children,Crime,Comedy,Western,Mystery,IMAX,Musical,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,0,1,0,0,0,0,...,0.04050,0.01425,0.03050,0.03500,0.14125,0.05775,0.03900,0.02975,0.08475,0.02200
2,0,0,1,1,0,0,0,0,0,0,...,0.05250,0.01575,0.01250,0.02000,0.12225,0.03275,0.02100,0.01100,0.10525,0.01975
3,0,0,0,0,0,1,0,0,0,0,...,0.06275,0.01950,0.02225,0.02300,0.12200,0.03475,0.01700,0.01800,0.09100,0.01775
4,0,0,0,0,0,1,0,0,0,0,...,0.05325,0.02800,0.01675,0.03875,0.18200,0.07050,0.01625,0.01425,0.08850,0.01500
5,0,0,0,0,0,1,0,0,0,0,...,0.05350,0.02050,0.01425,0.02550,0.19225,0.02675,0.01625,0.01300,0.08700,0.01600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0,0,0,0,0,0,0,0,0,0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
209159,0,0,0,0,0,0,0,0,0,0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
209163,0,0,0,0,0,1,0,0,0,0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [6]:
offline_sample = pd.read_csv(file_path1+'offline_sample.csv')
online_sample = pd.read_csv(file_path1+'online_sample.csv')

### Count the distinct users in offline sample

In [6]:
# Count the number of distinct users
n_users = rating_df['userId'].nunique()
print("Number of distinct users rating:", n_users)
n_users = offline_sample['userId'].nunique()
print("Number of distinct users offline:", n_users)
n_users = online_sample['userId'].nunique()
print("Number of distinct users online:", n_users)


Number of distinct users rating: 162541
Number of distinct users offline: 162541
Number of distinct users online: 162541


So in offline sample we have 162541 of distinct users

The data is still too large, so I managed to run 1000 users which consumed around 10 mins

In [7]:
sorted_offline_sample = offline_sample.sort_values('userId')

In [8]:
first_1000_users_data = sorted_offline_sample[sorted_offline_sample['userId'].between(1, 1000)]

In [9]:
first_1000_users_data
first_1000_users_data = first_1000_users_data.drop('timestamp', axis=1)

In [10]:
import time
#num_users = 1000  # Number of user profiles to include

user_profiles = []  # List to store individual user profiles
unique_user_ids = first_1000_users_data['userId'].unique()

start_time = time.time()  # Start measuring execution time

for user_id in unique_user_ids[:num_users]:
    # Step 1: Extract ratings for the current user
    user_ratings = first_1000_users_data[first_1000_users_data['userId'] == user_id]

    # Step 2: Left join user ratings with item_profiles_df
    combined_df = user_ratings.merge(item_profiles_df, left_on='movieId', right_index=True, how='left')

    # Step 3: Multiply ratings by movie features values
    for column in item_profiles_df.columns:
        combined_df[column] = combined_df[column] * combined_df['rating']

    # Step 4: Calculate sum of multiplied ratings for the current user
    user_profile = combined_df.sum()

    # Step 5: Divide sum of multiplied ratings by the number of movies for the current user
    num_movies = user_ratings['movieId'].nunique()
    user_profile = user_profile.div(num_movies)

    # Append the user profile to the list
    user_profiles.append(user_profile)

# Concatenate all user profiles into a single dataframe
user_profiles_df = pd.concat(user_profiles, axis=1).T
user_profiles_df = user_profiles_df.drop(columns=['timestamp', 'movieId'])

# Reset the index to be the user ID
user_profiles_df = user_profiles_df.set_index('userId')

end_time = time.time()  # Stop measuring execution time
execution_time = end_time - start_time

print("Execution time: {:.2f} seconds".format(execution_time))

# Print the resulting user_profiles_df
user_profiles_df

NameError: name 'num_users' is not defined

In [38]:
user_profiles_df = user_profiles_df.drop("rating", axis=1)

In [7]:
#user_profiles_df.to_csv('user_profiles.csv')
user_profiles_df = pd.read_csv(file_path1+'user_profiles.csv')

In [12]:
user_profiles_df

,userId,Thriller,Film-Noir,Fantasy,Children,Crime,Comedy,Western,Mystery,IMAX,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1.0,0.311594,0.050725,0.210145,0.115942,0.471014,1.289855,0.036232,0.202899,0.000000,...,0.396466,0.219810,0.221745,0.566649,0.857487,0.467949,0.264527,0.190792,0.327870,0.084998
1,2.0,0.628415,0.000000,0.603825,0.500000,0.308743,1.114754,0.068306,0.161202,0.155738,...,0.485318,0.159754,0.240888,0.185450,0.677732,0.416428,0.135429,0.204181,0.355231,0.084279
2,3.0,1.333849,0.033230,0.444359,0.275116,0.781298,0.934312,0.046368,0.355487,0.461360,...,0.487989,0.123478,0.136372,0.184707,0.846350,0.492728,0.197080,0.120515,0.478320,0.142230
3,4.0,0.810127,0.000000,0.459916,0.373418,0.620253,1.234177,0.107595,0.276371,0.386076,...,0.441826,0.129296,0.174878,0.152479,0.667380,0.423214,0.195332,0.136674,0.451525,0.154294
4,5.0,0.928571,0.000000,0.285714,0.306122,0.448980,1.693878,0.153061,0.285714,0.122449,...,0.330778,0.092441,0.125247,0.197732,0.693673,0.357533,0.105855,0.090176,0.365028,0.078661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.0,0.773333,0.000000,0.106667,0.053333,0.333333,0.973333,0.053333,0.000000,0.053333,...,0.583833,0.419100,0.656537,0.122757,0.630017,0.327893,0.163067,0.649350,0.376337,0.078623
996,997.0,0.958781,0.016316,0.307213,0.196007,0.621297,1.092744,0.037999,0.259553,0.158437,...,0.371166,0.109643,0.109484,0.158949,0.686094,0.353331,0.126022,0.105443,0.362272,0.099932
997,998.0,0.894737,0.000000,1.105263,0.684211,0.315789,1.684211,0.000000,0.157895,0.000000,...,0.247362,0.076441,0.068882,0.116526,0.652474,0.386362,0.144697,0.074664,0.398250,0.090467
998,999.0,0.893417,0.009404,0.338558,0.208464,0.528213,1.324451,0.056426,0.257053,0.040752,...,0.415904,0.126506,0.181029,0.252698,0.810908,0.442542,0.118147,0.137176,0.371747,0.082392


In [13]:
user_profiles_df = user_profiles_df.set_index('userId')
user_profiles_df 

,Thriller,Film-Noir,Fantasy,Children,Crime,Comedy,Western,Mystery,IMAX,Musical,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
userId,,,,,,,,,,,,,,,,,,,,,
1.0,0.311594,0.050725,0.210145,0.115942,0.471014,1.289855,0.036232,0.202899,0.000000,0.268116,...,0.396466,0.219810,0.221745,0.566649,0.857487,0.467949,0.264527,0.190792,0.327870,0.084998
2.0,0.628415,0.000000,0.603825,0.500000,0.308743,1.114754,0.068306,0.161202,0.155738,0.196721,...,0.485318,0.159754,0.240888,0.185450,0.677732,0.416428,0.135429,0.204181,0.355231,0.084279
3.0,1.333849,0.033230,0.444359,0.275116,0.781298,0.934312,0.046368,0.355487,0.461360,0.033230,...,0.487989,0.123478,0.136372,0.184707,0.846350,0.492728,0.197080,0.120515,0.478320,0.142230
4.0,0.810127,0.000000,0.459916,0.373418,0.620253,1.234177,0.107595,0.276371,0.386076,0.107595,...,0.441826,0.129296,0.174878,0.152479,0.667380,0.423214,0.195332,0.136674,0.451525,0.154294
5.0,0.928571,0.000000,0.285714,0.306122,0.448980,1.693878,0.153061,0.285714,0.122449,0.265306,...,0.330778,0.092441,0.125247,0.197732,0.693673,0.357533,0.105855,0.090176,0.365028,0.078661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.0,0.773333,0.000000,0.106667,0.053333,0.333333,0.973333,0.053333,0.000000,0.053333,0.000000,...,0.583833,0.419100,0.656537,0.122757,0.630017,0.327893,0.163067,0.649350,0.376337,0.078623
997.0,0.958781,0.016316,0.307213,0.196007,0.621297,1.092744,0.037999,0.259553,0.158437,0.053456,...,0.371166,0.109643,0.109484,0.158949,0.686094,0.353331,0.126022,0.105443,0.362272,0.099932
998.0,0.894737,0.000000,1.105263,0.684211,0.315789,1.684211,0.000000,0.157895,0.000000,0.000000,...,0.247362,0.076441,0.068882,0.116526,0.652474,0.386362,0.144697,0.074664,0.398250,0.090467


## According to you what could be the nice approach (classification or regression)? motivate your answer

According to the problem statement, the goal is to recognize user ratings based on the offline sample data. In this context, a suitable approach would be regression rather than classification. Regression models can predict the numerical ratings on a continuous scale, which aligns well with the task of estimating user ratings. Classification models, on the other hand, are more appropriate when the target variable has discrete categories or labels. However, since user ratings are typically represented as continuous values (e.g., 1-5 stars), regression is a more natural choice.


Input Data (Features):
- The input data should be represented as a 2-dimensional array or a pandas DataFrame.
- Each row represents an instance (sample) from the offline sample dataset.
- Each column represents a feature (input variable) that is used to predict the user rating.
- The values in the input data should be numerical or can be transformed into numerical representations suitable for scikit-learn models.

Output Data (Target Variable):
- The output data should be represented as a 1-dimensional array or a pandas Series.
- It represents the user rating, which is a continuous numerical value.
- The values in the output data should be continuous numerical ratings, such as 1-5 stars.


Now I want to predict the ratings by building a regression model.

In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between user_profiles_df and item_profiles_df
similarity_matrix = cosine_similarity(user_profiles_df, item_profiles_df)

# Scale the similarity values to a rating scale of 1-5
#min_value = similarity_matrix.min()
#max_value = similarity_matrix.max()
#scaled_ratings = ((similarity_matrix - min_value) / (max_value - min_value)) * 4 + 1

# Create a dataframe with scaled ratings and set the movieId as the index
similarity_df = pd.DataFrame(similarity_matrix , index=user_profiles_df.index, columns=item_profiles_df.index)

# Print the resulting scaled_ratings_df
similarity_df


movieId,1,2,3,4,5,6,7,8,9,10,...,209145,209147,209151,209153,209155,209157,209159,209163,209169,209171
userId,,,,,,,,,,,,,,,,,,,,,
1.0,0.712344,0.596159,0.696979,0.710661,0.626517,0.785555,0.686869,0.629586,0.546440,0.598231,...,0.115682,0.014365,0.000000,0.117313,0.058807,0.115682,0.001129,0.117313,0.000000,0.084382
2.0,0.844725,0.784948,0.790805,0.741640,0.721326,0.779738,0.751910,0.750787,0.677594,0.765549,...,0.072205,0.010761,0.000000,0.083105,0.069971,0.072205,0.000000,0.083105,0.000000,0.109286
3.0,0.787478,0.757388,0.781907,0.730525,0.706299,0.811957,0.728326,0.726221,0.722177,0.783937,...,0.057660,0.030481,0.000228,0.068605,0.075546,0.057660,0.000619,0.068605,0.000228,0.105622
4.0,0.820973,0.756194,0.772689,0.702948,0.692236,0.811673,0.713522,0.713244,0.702187,0.783829,...,0.032081,0.022496,0.000000,0.059547,0.080468,0.032081,0.003654,0.059547,0.000000,0.100894
5.0,0.822709,0.754190,0.817832,0.775904,0.743281,0.803700,0.766949,0.748634,0.674774,0.743092,...,0.072416,0.018797,0.000000,0.105005,0.080444,0.072416,0.000000,0.105005,0.000000,0.080973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.0,0.784595,0.753518,0.779433,0.737909,0.705375,0.785681,0.736164,0.716815,0.720329,0.776827,...,0.071340,0.016815,0.000000,0.080384,0.077354,0.071340,0.002320,0.080384,0.000000,0.105817
997.0,0.784177,0.746060,0.808447,0.771078,0.731631,0.824054,0.754967,0.740395,0.717980,0.770189,...,0.081331,0.034677,0.001707,0.102365,0.087696,0.081331,0.003190,0.102365,0.001707,0.105634
998.0,0.870413,0.806180,0.819727,0.758913,0.755402,0.745303,0.779018,0.747073,0.684510,0.763901,...,0.038601,0.009098,0.000000,0.075820,0.095336,0.038601,0.000000,0.075820,0.000000,0.079240


In [15]:
# Reset index of similarity_df to make user IDs a column
similarity_df = similarity_df.reset_index()

# Melt similarity_df to convert column names (movieId) to a new column
similarity_df = similarity_df.melt(id_vars='userId', var_name='movieId', value_name='similarity')

# Convert movieId column to numeric type
similarity_df['movieId'] = similarity_df['movieId'].astype(int)

# Merge rating_df with similarity_df based on 'userId' and 'movieId'
combined_df = first_1000_users_data.merge(similarity_df, on=['userId', 'movieId'], how='left')

# Print the combined dataframe
combined_df


,userId,movieId,rating,similarity
0,1,2692,5.0,0.889585
1,1,8786,4.0,0.828213
2,1,27266,4.5,0.834901
3,1,4325,5.0,0.818635
4,1,6016,5.0,0.861949
...,...,...,...,...
140680,1000,253,4.0,0.803222
140681,1000,1265,2.0,0.869213
140682,1000,2467,3.5,0.847423
140683,1000,778,4.5,0.865398


In [8]:
#combined_df.to_csv('combined.csv')
combined_df = pd.read_csv(file_path1+'combined.csv')

We are going to use combined_df to train the model with linear regression

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [12]:
from sklearn.model_selection import train_test_split

# Create the feature matrix X and the target vector y
X = combined_df[['userId', 'movieId','similarity']]
y = combined_df['rating']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the training and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Train set shape: (112548, 3) (112548,)
Test set shape: (28137, 3) (28137,)


In [ ]:
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
# Initialize the model
model = LinearRegression()

# Perform cross-validation and calculate the mean squared error
scores = cross_val_score(model, X_train, y_train, cv=5)

model.fit(X_train,y_train)

test_score = model.score(X_test,y_test)
print(scores)
print(test_score)


So it's finally seems like my kernel is crushed on this cell, the following steps should be match the online sample with item_profiles_df and user_profiles_df as well and use the model we trained to predict the ratings, and then use the following evaluation to evaluate the well recommended movies.    
As we did before in the last submission.

In [ ]:
total_users = len(online_sample['userId'].unique())
total_well_recommended_movies = 0

for user_id in online_sample['userId'].unique():
    user_data = online_sample[online_sample['userId'] == user_id]
    positively_rated_movies = len(user_data[user_data['rating'] > 2])

    # Check if there are positively rated movies for the user
    if positively_rated_movies > 0:
        well_recommended_movies = len(user_data[(user_data['predicted_rating'] > 2) & (user_data['rating'] > 2)])
        proportion = well_recommended_movies / positively_rated_movies
        total_well_recommended_movies += proportion

evaluation_model = total_well_recommended_movies / total_users
print("Evaluation Model:", evaluation_model)
